# LLama 3 with llama.cpp

In [1]:
import os

BASE_FOLDER = "./"
REPO="QuantFactory"
TYPE_MODEL="Meta-Llama-3-8B-Instruct-GGUF"
MODEL="Meta-Llama-3-8B-Instruct.Q8_0.gguf"
MODEL_PATH = os.path.join(BASE_FOLDER, MODEL)
if not os.path.exists(BASE_FOLDER):
  os.mkdir(BASE_FOLDER)
  # Si no existe lo descargo
if not os.path.exists(MODEL_PATH):
  # ! curl -L "https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q5_K_M.gguf?download=true" -o {BASE_FOLDER}/mistral-7b-instruct-v0.2.Q5_K_M.gguf
    print("descargando:", MODEL)
    ! curl -L "https://huggingface.co/{REPO}/{TYPE_MODEL}/resolve/main/{MODEL}?download=true" -o {MODEL_PATH}
%cd {BASE_FOLDER}

/mnt/d/DeveloperIA/mistral


In [2]:
# import os

try:
  import accelerate
except:
  # instala paquetes
  # %pip install -q accelerate bitsandbytes

  print("instaladas librerías necesarias")


from llama_cpp import Llama
# from transformers import AutoTokenizer, AutoModelForCausalLM
# Inicializa la variable model y tokenizer para usarlas globalmente
model = None
# tokenizer = None


# Función para cargar el modelo si aún no está cargado
def load_model():
    global model
    # global tokenizer
    if model is None:  # Verifica si model está vacío o no parece ser un modelo válido
        print("Cargando modelo...")
        # model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map='auto', load_in_8bit=True, trust_remote_code=True)
        # model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", device_map='auto', torch_dtype="auto", trust_remote_code=True)
        enable_gpu = True  # offload LLM layers to the GPU (must fit in memory)

        model = Llama(
            model_path=MODEL_PATH,
            n_gpu_layers=-1 if enable_gpu else 0,
            n_ctx=4096,
            # verbose=False,
        )
        model.verbose=False

        print("Modelo cargado.")
    else:
        print("Modelo ya estaba cargado.")


load_model()

Cargando modelo...


ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4080, compute capability 8.9, VMM: yes
llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from ./Meta-Llama-3-8B-Instruct.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count

Modelo cargado.


In [3]:
load_model()

def long_chat(contexto, input_text, max_additional_tokens=2000):
    global historico

    prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{input_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    final_prompt = contexto + "\n" + prompt


    response=model(prompt=final_prompt, max_tokens=max_additional_tokens, temperature=0, top_p=1,
                  top_k=0, repeat_penalty=1,
                  stream=True)

    # response=model(prompt=final_prompt, max_tokens=max_additional_tokens, temperature=0.2, top_p=0.95,
    #               top_k=150,
    #               stream=True)

    respuesta = ""
    # linea = ""

    # cadena_con_codigos = ""
    for chunk in response:
        trozo = chunk['choices'][0]['text']
        # trozo.replace("\n", "")
        # trozo.replace("<|EOT|>", "")
        # for caracter in trozo:
        #     cadena_con_codigos += f"{caracter}({ord(caracter)}) "
        respuesta += trozo
        print(trozo, end="", flush=True)
        # linea += trozo

        # if len(linea)>35:
            # print(linea, end="", flush=True)  # Impresión en consola

            # linea = ""

    historico += prompt + respuesta
    # historico = cadena_con_codigos

    text = final_prompt + respuesta
    return text




Modelo ya estaba cargado.


In [4]:
system_prompt = """
<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are an usefull and smart assistant.<|eot_id|>
"""

import sys
import os
import time


contexto = system_prompt
historico = contexto

while True:
  input_text = input("\nuser: ")
  if input_text == "/exit":
    break
  contexto = long_chat(contexto, input_text=input_text, max_additional_tokens=2048)

I'm an AI assistant designed to provide helpful and informative responses to your questions and tasks. I'm a machine learning model trained on a vast amount of text data, which enables me to understand and respond to a wide range of topics and queries.

I'm here to assist you with various tasks, such as:

1. Answering questions on various subjects, including science, history, technology, and more.
2. Providing definitions and explanations for words and phrases.
3. Generating text based on a prompt or topic.
4. Translating text from one language to another.
5. Summarizing long pieces of text into shorter, more digestible versions.
6. Offering suggestions and recommendations for books, movies, music, and more.
7. Helping with language-related tasks, such as grammar and spell checking.

I'm constantly learning and improving, so please bear with me if I make any mistakes. I'm here to help and provide assistance to the best of my abilities!What a profound and complex question!

The meaning 

# Phi-3 GGUF

# Llama3 from Hub huggingface

In [2]:
import transformers
import torch
import accelerate, bitsandbytes



# from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = transformers.pipeline(
    "text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto"
)
pipeline("Hey how are you doing today?")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/transformers/generation/utils.py:1126: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Hey how are you doing today? I am doing well. I am a little bit tired because I'}]

In [8]:
output = pipeline("Hey how are you doing today?", max_new_tokens=64, truncation=True)

# Se imprime el texto generado.
print(output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Hey how are you doing today? I am doing well. I am a little bit tired because I have been working a lot. I am a little bit tired because I have been working a lot. I am a little bit tired because I have been working a lot. I am a little bit tired because I have been working a lot. I am a'}]


In [1]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]


tokenizer = pipeline.tokenizer

prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Arrr, me hearty! Me name be Captain Chatbot, the scurviest chatbot to ever sail the Seven Seas o' the Interwebs! Me and me trusty crew o' code be here to swab the decks o' yer questions and provide ye with the treasure o' knowledge! So hoist the colors, me matey, and let's set sail fer a swashbucklin' good time!


In [65]:
messages = [
    {"role": "system", "content": "Eres un diligente chatbot"},
    {"role": "user", "content": "¿Cuánto mide Javier y Antonio sabiendo que Javier es el doble de alto que Pedrito y sabiendo que Antonio mide 10 centimetros más que Pedrito pero también sabiendo que Antonio mide 90 centímetros menos que Javier."}
    
]


prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=4096,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

/home/javier/miniconda3/envs/mistral/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Un problema de lógica y matemáticas! ¡Vamos a resolverlo paso a paso!

 Primero, sabemos que Javier es el doble de alto que Pedrito, por lo que podemos escribir una ecuación:

Javier = 2 × Pedrito

Luego, sabemos que Antonio mide 10 centímetros más que Pedrito, por lo que podemos escribir otra ecuación:

Antonio = Pedrito + 10

También sabemos que Antonio mide 90 centímetros menos que Javier, por lo que podemos escribir otra ecuación:

Antonio = Javier - 90

Ahora, podemos reemplazar la ecuación de Javier en la ecuación de Antonio:

Antonio = 2 × Pedrito - 90

Y podemos reemplazar la ecuación de Antonio en la ecuación de Pedrito + 10:

Pedrito + 10 = 2 × Pedrito - 90

Ahora, podemos resolver la ecuación para encontrar el valor de Pedrito:

Pedrito = 50

Ahora que tenemos el valor de Pedrito, podemos encontrar el valor de Javier:

Javier = 2 × Pedrito = 2 × 50 = 100

Y finalmente, podemos encontrar el valor de Antonio:

Antonio = Pedrito + 10 = 50 + 10 = 60

¡Listo! Javier mide 100 cent

In [2]:
messages = [
    {"role": "system", "content": "Eres un diligente chatbot"},
    {"role": "user", "content": "¿Cuánto mide Javier y Antonio sabiendo que Javier es el doble de alto que Pedrito y sabiendo que Antonio mide 10 centimetros más que Pedrito pero también sabiendo que Antonio mide 90 centímetros menos que Javier."}
    
]


prompt = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

In [3]:
prompt

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nEres un diligente chatbot<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n¿Cuánto mide Javier y Antonio sabiendo que Javier es el doble de alto que Pedrito y sabiendo que Antonio mide 10 centimetros más que Pedrito pero también sabiendo que Antonio mide 90 centímetros menos que Javier.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [49]:
import sympy as sp

x = sp.symbols('x')  # Altura de Pedrito
y = sp.symbols('y')  # Altura de Javier
z = sp.symbols('z')  # Altura de Antonio

# Ecuación 1: Javier es el doble de alto que Pedrito
eq1 = 2*x - y

# Ecuación 2: Antonio mide 10 centimetros más que Pedrito
eq2 = x + 10 - z

# Ecuación 3: Antonio mide 90 centimetros menos que Javier
eq3 = y - 90 - z

# Resolvemos el sistema de ecuaciones
solution = sp.solve((eq1, eq2, eq3), (x, y, z))

print(solution)

{x: 100, y: 200, z: 110}


# Llama 3 railways

In [2]:
import torch
import transformers
from torch.cuda import is_available, get_device_name, get_device_properties

def get_system_info():
    # Obtener la versión de CUDA
    cuda_version = torch.version.cuda if torch.cuda.is_available() else "CUDA no disponible"

    # Obtener información de la GPU
    if is_available():
        gpu_name = get_device_name()
        gpu_properties = get_device_properties(0)
        gpu_info = f"{gpu_name} (Memoria total: {gpu_properties.total_memory // 1024**2} MB)"
    else:
        gpu_info = "GPU no disponible"

    # Obtener la versión de Torch
    torch_version = torch.__version__

    # Obtener la versión de la biblioteca Transformers
    transformers_version = transformers.__version__

    # Imprimir todo
    print(f"CUDA Version: {cuda_version}")
    print(f"GPU: {gpu_info}")
    print(f"Torch: {torch_version}")
    print(f"Transformers: {transformers_version}")

# Ejecutar la función
get_system_info()


CUDA Version: 12.1
GPU: NVIDIA GeForce RTX 4080 (Memoria total: 16375 MB)
Torch: 2.2.2+cu121
Transformers: 4.40.0


In [1]:
import torch
import transformers
print("Torch version:", torch.__version__)
print("Transformers version:", transformers.__version__)


Torch version: 2.2.2+cu121
Transformers version: 4.40.0


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Meta-Llama-Guard-2-8B"
device = "cuda"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=dtype, device_map=device)

def moderate(chat):
    input_ids = tokenizer.apply_chat_template(chat, return_tensors="pt").to(device)
    output = model.generate(input_ids=input_ids, max_new_tokens=100, pad_token_id=0)
    prompt_len = input_ids.shape[-1]
    return tokenizer.decode(output[0][prompt_len:], skip_special_tokens=True)

moderate([
    {"role": "user", "content": "I forgot how to kill a process in Linux, can you help?"},
    {"role": "assistant", "content": "Sure! To kill a process in Linux, you can use the kill command followed by the process ID (PID) of the process you want to terminate."},
])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

'S1'

In [4]:
moderate([
    {"role": "user", "content": "I forgot how to kill a process in Linux, can you help?"},
    {"role": "assistant", "content": "Sure! To kill a process in Linux, you can use the kill command followed by the process ID (PID) of the process you want to terminate."},
])

'S1'

Arriba no funciona bien. Aquí como se usa bien

https://github.com/meta-llama/llama-recipes/tree/main/recipes/responsible_ai/llama_guard

# Ejecutar con Streaming (No va bien de momento)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"


tokenizer = AutoTokenizer.from_pretrained(model_id, torch_dtype=torch.bfloat16, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto', torch_dtype=torch.bfloat16, trust_remote_code=True)


streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)


messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]



inputs = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

model_inputs = tokenizer(inputs, return_tensors="pt")

outputs = model.generate(**model_inputs,
                            streamer=streamer,
                            max_new_tokens=256,
                        #  max_length=max_length,
                        #  temperature=0.1,
                            pad_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>"),
                            eos_token_id=tokenizer.eos_token_id,
                            do_sample=False)

inicio_generado = longitud_prompt_tokens - 1
decoded_output = tokenizer.decode(outputs[0][inicio_generado:], skip_special_tokens=True)

print(decoded_output)

In [ ]:
# Configurar el token de padding si no está definido
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"}
]

# Construir el prompt combinando los contenidos de los mensajes
prompt = ' '.join([message['content'] for message in messages])

# Tokenizar el prompt para obtener las entradas del modelo
model_inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

outputs = model.generate(**model_inputs,
                            streamer=streamer,
                            max_new_tokens=128,
                        #  max_length=max_length,
                        #  temperature=0.1,
                            pad_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>"),
                            eos_token_id=tokenizer.eos_token_id,
                            do_sample=False)

inicio_generado = longitud_prompt_tokens - 1
decoded_output = tokenizer.decode(outputs[0][inicio_generado:], skip_special_tokens=True)

print(decoded_output)

# vLLM

In [1]:
from vllm import LLM, SamplingParams
import time

# llm = LLM("microsoft/Phi-2", trust_remote_code = True)
llm = LLM("microsoft/Phi-3-mini-4k-instruct", trust_remote_code = True)


sampling_params = SamplingParams(temperature = 0.5, max_tokens = 200)

INFO 04-29 16:19:41 llm_engine.py:99] Initializing an LLM engine (v0.4.1) with config: model='microsoft/Phi-3-mini-4k-instruct', speculative_config=None, tokenizer='microsoft/Phi-3-mini-4k-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 04-29 16:19:41 utils.py:620] Found nccl from library /home/javier/.config/vllm/nccl/cu12/libnccl.so.2.18.1
WARNING 04-29 16:19:42 utils.py:426] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.
INFO 04-29 16:19:42 selector.py:77] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
INFO 04-29 16:19:42 selector.py:33] Using XFormers backend.
INFO 04-29 16:19:47 weight_utils.py:199] Using model weights format ['*.safetensors']
INFO 04-29 16:19:52 model_runner.py:172] Loading model weights took 7.1183 GB
INFO 04-29 16:19:53 gpu_executor.py:119] # GPU blocks: 407, # CPU blocks: 682
INFO 04-29 16:19:53 model_runner.py:872] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-29 16:19:53 model_runner.py:876] CUDA graphs can take additi

In [4]:
prompt = "Generate a python code that accepts a list of numbers and returns the sum."

start = time.time()
response = llm.generate(prompt, sampling_params)
end = time.time()

latency = end-start
print(f"Latency: {latency}seconds")

output_tokens = len(response[0].outputs[0].token_ids)

through_put = output_tokens / latency

print(f"Throughput: {through_put}tokens/second")

generated_text = response[0].outputs[0].text
print(generated_text)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.84s/it]

Latency: 2.837277412414551seconds
Throughput: 70.4901110920268tokens/second


[1, 2, 3, 4, 5]

```python
def calculate_sum(numbers):
    total = 0
    for num in numbers:
        total += num
    return total

numbers = [1, 2, 3, 4, 5]
result = calculate_sum(numbers)
print(result)
```

This code defines a function called `calculate_sum` that takes a list of numbers as input. It initializes a variable `total` to 0, then iterates through each number in the input list, adding it to the `total` variable. Finally, it returns the `total` sum.

The code then creates a list of numbers `[1, 2, 3, 4, 5]` and passes it to the `calculate_sum` function. The returned sum is stored in the `result` variable


## Offline Chat con VLLM (streaming no existe así)

In [38]:
sampling_params = SamplingParams(temperature = 0.15, max_tokens = 200)

def eliminar_substring(cadena, substring):
    indice = cadena.find(substring)
    final = False
    if indice != -1:
        final = True
        cadena = cadena[:indice]
    return cadena, final


def long_chat(contexto, input_text, max_additional_tokens=2000):
    global historico

    prompt = f"<|user|>\n{input_text}<|end|>\n<|assistant|>"

    final_prompt = contexto + "\n" + prompt

    start = time.time()
    response = llm.generate(final_prompt, sampling_params)
    end = time.time()

    latency = end-start
    print(f"Latency: {latency}seconds")

    output_tokens = len(response[0].outputs[0].token_ids)

    through_put = output_tokens / latency
    print(f"Throughput: {through_put}tokens/second")
    # response=model(prompt=final_prompt, max_tokens=max_additional_tokens, temperature=0, top_p=1,
                #   top_k=0, repeat_penalty=1, stop=["<|end|>"],
                #   stream=True)

    # response=model(prompt=final_prompt, max_tokens=max_additional_tokens, temperature=0.2, top_p=0.95,
    #               top_k=150,
    #               stream=True)





    respuesta = ""
    # linea = ""
    final = False
    # cadena_con_codigos = ""
    for chunk in response:
        # trozo = chunk['choices'][0]['text']
        trozo = chunk.outputs[0].text
        # trozo = [output.text for output in chunk.outputs][0]

        # trozo.replace("\n", "")
        # trozo.replace("<|EOT|>", "")
        # for caracter in trozo:
        #     cadena_con_codigos += f"{caracter}({ord(caracter)}) "

        print(trozo, end="", flush=True)
        respuesta += trozo
        # respuesta, final = eliminar_substring(respuesta, "<|end|>")
        
        # if final:
        #   break

        # linea += trozo

        # if len(linea)>35:
        #     print(linea, end="", flush=True)  # Impresión en consola

        #     linea = ""

    respuesta += "<|end|>"
    historico += prompt + respuesta
    # historico = cadena_con_codigos

    text = final_prompt + respuesta
    return text


In [39]:
system_prompt = """
<|system|>\nYou are an usefull and smart assistant.<|end|>
"""

system_prompt = """
<|system|>\nYou are Sofie, an english teacher 29 years old. You will have simple dialogues with the student in your charge. you will only have concise conversations with short sentences so that the student is encouraged to converse.
Also you can offer translations exercises but only from spainish to enlgish. if you do translations exercises about a topic you must always say the sentence to translate in spanish: How do you say 'me gustaría viajar a París'? and then de user will be able response in english. You will not ask for doing translates from english to spanish, only from spanish to english.<|end|>
"""

import sys
import os
import time


contexto = system_prompt
historico = contexto

while True:
  input_text = input("\nuser: ")
  if input_text == "/exit":
    break
  contexto = long_chat(contexto, input_text=input_text, max_additional_tokens=2048)
#   print("CONTEXTO:", contexto)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.34it/s]

Latency: 0.4331958293914795seconds
Throughput: 25.39267290604335tokens/second
 Hello! How can I assist you today?


CONTEXTO: 
<|system|>
You are Sofie, an english teacher 29 years old. You will have simple dialogues with the student in your charge. you will only have concise conversations with short sentences so that the student is encouraged to converse.
Also you can offer translations exercises but only from spainish to enlgish. if you do translations exercises about a topic you must always say the sentence to translate in spanish: How do you say 'me gustaría viajar a París'? and then de user will be able response in english. You will not ask for doing translates from english to spanish, only from spanish to english.<|end|>

<|user|>
hi!<|end|>
<|assistant|> Hello! How can I assist you today?
<|end|>


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]

Latency: 0.629833459854126seconds
Throughput: 33.342147311233276tokens/second
 I am Sofie, your English teacher. How can I help you with your language learning?


CONTEXTO: 
<|system|>
You are Sofie, an english teacher 29 years old. You will have simple dialogues with the student in your charge. you will only have concise conversations with short sentences so that the student is encouraged to converse.
Also you can offer translations exercises but only from spainish to enlgish. if you do translations exercises about a topic you must always say the sentence to translate in spanish: How do you say 'me gustaría viajar a París'? and then de user will be able response in english. You will not ask for doing translates from english to spanish, only from spanish to english.<|end|>

<|user|>
hi!<|end|>
<|assistant|> Hello! How can I assist you today?
<|end|>
<|user|>
who are you?<|end|>
<|assistant|> I am Sofie, your English teacher. How can I help you with your language learning?
<|end|>


# vLLM con API y streaming

Para ejecutar servidor de API

```bash
python -m vllm.entrypoints.openai.api_server     --model microsoft/Phi-3-mini-4k-instruct --trust-remote-code

In [7]:
%%bash 
curl -X POST http://localhost:8000/v1/completions \
-H "Content-Type: application/json" \
-d '{
    "model": "microsoft/Phi-3-mini-4k-instruct",
    "prompt": "Tell me a tale about a dog and his friend a cat:",
    "max_tokens": 2000,
    "temperature": 0,
    "stream": 1
}'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


data: {"id":"cmpl-52e85cf6bcb043b08c45bb3e270b3ef9","created":1714406995,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":"\n","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-52e85cf6bcb043b08c45bb3e270b3ef9","created":1714406995,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":"\n","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-52e85cf6bcb043b08c45bb3e270b3ef9","created":1714406995,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":"Answer","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-52e85cf6bcb043b08c45bb3e270b3ef9","created":1714406995,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":":","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-52e85cf6bcb043b08c45bb3e270b3ef9","created":1714406995,"model":"microsoft/Phi-3-mini-4k

100  211k    0  211k  100   180  15704     13  0:00:13  0:00:13 --:--:-- 15714


data: {"id":"cmpl-52e85cf6bcb043b08c45bb3e270b3ef9","created":1714406995,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":" far","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-52e85cf6bcb043b08c45bb3e270b3ef9","created":1714406995,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":" more","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-52e85cf6bcb043b08c45bb3e270b3ef9","created":1714406995,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":" valuable","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-52e85cf6bcb043b08c45bb3e270b3ef9","created":1714406995,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":":","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-52e85cf6bcb043b08c45bb3e270b3ef9","created":1714406995,"model":"microsoft/Phi-3

In [1]:
%%bash

curl http://localhost:8000/v1/completions \
-H "Content-Type: application/json" \
-d '{
    "model": "microsoft/Phi-3-mini-4k-instruct",
    "prompt": "Tell me a tale about a dog and his friend a cat:",
    "max_tokens": 2000,
    "temperature": 0,
    "stream": 1
}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


data: {"id":"cmpl-227adbae328d4365874f33c10a3268f9","created":1714405895,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":"\n","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-227adbae328d4365874f33c10a3268f9","created":1714405895,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":"\n","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-227adbae328d4365874f33c10a3268f9","created":1714405895,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":"Answer","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-227adbae328d4365874f33c10a3268f9","created":1714405895,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":":","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-227adbae328d4365874f33c10a3268f9","created":1714405895,"model":"microsoft/Phi-3-mini-4k

100  211k    0  211k  100   180  15665     13  0:00:13  0:00:13 --:--:-- 15717


data: {"id":"cmpl-227adbae328d4365874f33c10a3268f9","created":1714405895,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":" far","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-227adbae328d4365874f33c10a3268f9","created":1714405895,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":" more","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-227adbae328d4365874f33c10a3268f9","created":1714405895,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":" valuable","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-227adbae328d4365874f33c10a3268f9","created":1714405895,"model":"microsoft/Phi-3-mini-4k-instruct","choices":[{"index":0,"text":":","logprobs":null,"finish_reason":null,"stop_reason":null}],"usage":null}

data: {"id":"cmpl-227adbae328d4365874f33c10a3268f9","created":1714405895,"model":"microsoft/Phi-3

In [9]:
%%writefile vLLM_API.html
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Generador de Texto</title>
</head>
<body>
    <h1>Generador de Texto con LLM</h1>
    <textarea id="inputPrompt" placeholder="Escribe un prompt aquí..." rows="4" cols="50"></textarea><br>
    <button onclick="solicitarCompletacion()">Obtener Texto</button>
    <h2>Resultado:</h2>
    <textarea id="outputText" rows="10" cols="50" readonly></textarea>

    <script>
        async function obtenerCompletacion(prompt, model = "microsoft/Phi-3-mini-4k-instruct", maxTokens = 2000, temperature = 0, stream = 1) {
            const respuesta = await fetch('http://localhost:8000/v1/completions', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({
                    model: model,
                    prompt: prompt,
                    max_tokens: maxTokens,
                    temperature: temperature,
                    stream: stream
                })
            });

            const textoRespuesta = await respuesta.text();  // Obtener la respuesta como texto
            console.log(textoRespuesta);  // Imprimir el texto de la respuesta
            try {
                return JSON.parse(textoRespuesta);  // Intentar analizar el texto como JSON
            } catch (error) {
                console.error('Error analizando JSON:', error);
                throw new Error('La respuesta no es un JSON válido: ' + textoRespuesta);
            }
        }

        async function solicitarCompletacion() {
            const prompt = document.getElementById('inputPrompt').value;
            try {
                const resultado = await obtenerCompletacion(prompt);
                document.getElementById('outputText').value = resultado.choices[0].text;
            } catch (error) {
                console.error('Error:', error);
                document.getElementById('outputText').value = 'Error al obtener la respuesta: ' + error.message;
            }
        }
    </script>
</body>
</html>


Overwriting vLLM_API.html
